<a href="https://colab.research.google.com/github/alexjordyy/c1/blob/main/Twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
! pip install kaggle

In [88]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [89]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sentiment140


In [90]:
!kaggle datasets download -d kazanova/sentiment140 --force


Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 73% 59.0M/80.9M [00:00<00:00, 616MB/s]
100% 80.9M/80.9M [00:00<00:00, 357MB/s]


In [91]:
# extracting the file
from zipfile import ZipFile
dataset ='/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:#reading the zipfile with name zip
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [92]:
#installing dependencies
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer#steming used reduce the word to root word
from sklearn.feature_extraction.text import TfidfVectorizer#to extract the festures and TfidfVectorizer textual data to numeric data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression#ml model
from sklearn.metrics import accuracy_score#metrics to performance evaluation

In [93]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [94]:
import nltk
print(stopwords.words('english'))
#DOWNLOAD ALL STOPWORDS IN ENGLISH

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [95]:
#data processcing load data from csv to panda dataframe
data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

NOW DATA IS LOADED

In [96]:
data.shape#no of row & columns

(1599999, 6)

In [97]:
data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [98]:
data.isnull().sum()

,0
0,0
1467810369,0
Mon Apr 06 22:19:45 PDT 2009,0
NO_QUERY,0
_TheSpecialOne_,0
"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D",0


In [99]:
#name the col n read the dataset again[ load the Sentiment140 dataset and manually name its columns]
cname=['target','id','date','flag','user','text']
data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',names=cname)

In [100]:
data.shape

(1600000, 6)

In [101]:
data.head()


,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [102]:
#data refining check for null values
data.isnull().sum()#pandas as this

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [103]:
#distribution of the dataset like no of labels present
data['target'].value_counts()

,count
target,
0,800000
4,800000


as the labels are divided equally entropy is 1 .~. no uncertainity

In [104]:
from math import log2

# Calculate entropy function
def entropy(series):
    counts = series.value_counts(normalize=True)
    return -sum(p * log2(p) for p in counts)

# Apply to the sentiment column
ent = entropy(data['target'])
print(f"Entropy of sentiment labels: {ent:.4f}")

Entropy of sentiment labels: 1.0000


In [105]:
#convert  4 to 1
data.replace({'target': {4: 1}}, inplace=True)

In [106]:
#distribution of the dataset like no of labels present
data['target'].value_counts()

,count
target,
0,800000
1,800000


0 -ve tweet
1 +ve tweet

In [107]:
#stemming
#procees to reduce a word to its keyword or root word
#actor actress acting = act to process n refine the data
port_stem=PorterStemmer()


In [108]:
from nltk.stem.porter import PorterStemmer
port_stem = PorterStemmer()

def stemming(content):#tweets are passed
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  #sub function:lowercase n uppercase '^' remove the tweets which doesnt belong this alphabet list no numbers n special characters
  stemmed_content=stemmed_content.lower()#lowercase is sufficient
  stemmed_content=stemmed_content.split()#split the data or all the letters
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  #here we are getting the words after removing puncuations we also remove stopwords not do stemming
  stemmed_content=' '.join(stemmed_content)
  #we join the words inorder
  return stemmed_content

In [110]:
data['stemmed_content'] = data['text'].apply(stemming)



In [111]:
data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [112]:
print(data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [113]:
print(data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [114]:
#now model understands the relationship of 0 n 1
#separating the data n label
x=data['stemmed_content'].values
y=data['target'].values


In [115]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [116]:
print(y)

[0 0 0 ... 1 1 1]


In [117]:
#split data into test n train
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)
#stratify=y means equal proportion of classes


In [118]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [119]:
print(x_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [120]:
#conversion of textual data to numeric
vectorizer=TfidfVectorizer()
x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)

In [121]:
print(x_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453092 stored elements and shape (1280000, 461488)>
  Coords	Values
  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (12

In [122]:
print(x_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2289192 stored elements and shape (320000, 461488)>
  Coords	Values
  (0, 15110)	0.1719352837797837
  (0, 31168)	0.1624772418052177
  (0, 67828)	0.26800375270827315
  (0, 106069)	0.36555450010904555
  (0, 132364)	0.255254889555786
  (0, 138164)	0.23688292264071406
  (0, 171378)	0.2805816206356074
  (0, 271016)	0.45356623916588285
  (0, 279082)	0.17825180109103442
  (0, 388348)	0.2198507607206174
  (0, 398906)	0.34910438732642673
  (0, 409143)	0.3143047059807971
  (0, 420984)	0.17915624523539805
  (1, 6463)	0.30733520460524466
  (1, 15110)	0.211037449588008
  (1, 145393)	0.575262969264869
  (1, 217562)	0.40288153995289894
  (1, 256777)	0.28751585696559306
  (1, 348135)	0.4739279595416274
  (1, 366203)	0.24595562404108307
  (2, 22532)	0.3532582957477176
  (2, 34401)	0.37916255084357414
  (2, 89448)	0.36340369428387626
  (2, 183312)	0.5892069252021465
  (2, 256834)	0.2564939661498776
  :	:
  (319994, 443794)	0.2782185641032538


In [123]:
#train our data model logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [124]:
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score( y_train,x_train_prediction)

In [125]:
print(training_data_accuracy*100,'%')#for training data

79.87195312499999 %


In [126]:
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score( y_test,x_test_prediction)

In [127]:
print(test_data_accuracy*100,'%')

77.668125 %


In [128]:
#to save the trained model
import pickle

In [129]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))#dumping the model and its learning,u ll open the n we ll writing only n in binary fromat

In [130]:
#using the sanved model we ll predict the new values
l=pickle.load(open('trained_model.sav','rb'))

In [131]:
x_new=x_test[200]
print(y_test[200])
prediction=model.predict(x_new)
print(prediction)
if(prediction[0]==0):
  print('negative')
else:
  print('positive')

1
[1]
positive


In [132]:
x_new=x_test[3]
print('true value')
print(y_test[3])
print('predicted value')
prediction=model.predict(x_new)
print(prediction)

if(prediction[0]==0):
  print('negative')
else:
  print('positive')

true value
0
predicted value
[0]
negative
